# Module 2: Circuits and Connectivity

A quantum circuit is a sequence of gates acting on a register of qubits. While single-qubit physics is rich, the true complexity—and the primary source of error—arises from multi-qubit interactions and the constraints of the physical hardware.

## 2.1 The "Killer" Gate: Cross-Resonance (CR)

Single-qubit gates are fast and accurate (99.9%). The problem lies in the entangling gates (2-qubit gates).

**The Physics:** IBM hardware uses the **Cross-Resonance (CR)** effect. We drive **Qubit 0 (Control)** at the frequency of **Qubit 1 (Target)**.

**Mechanism:** Because the qubits are coupled, driving $Q_c$ at $\omega_t$ induces a driven oscillation on $Q_t$. Crucially, the *strength* and *sign* of this drive depend on the state of $Q_c$ ($|0\rangle$ or $|1\rangle$).

The effective Hamiltonian is approximately:
$$H_{CR} \approx ZX = \sigma_z^{(c)} \otimes \sigma_x^{(t)}$$

**The Cost:**
* **Slow:** CR gates are slow (300–600 ns) compared to single-qubit gates (30–60 ns). This exposes qubits to significant decoherence ($T_1/T_2$).
* **Leaky:** The strong drive can cause **crosstalk** (errors on neighbors).

**Competition Insight:** In the AQC competition, identifying the number and location of ECR/CNOT gates is the single most predictive feature for total error.

## 2.2 Topology and SWAPs

Qubits are not fully connected; they are laid out in a grid, often a **Heavy-Hex lattice**. You cannot entangle Qubit 0 and Qubit 10 directly.

**SWAP Gate:** To interact distant qubits, the compiler inserts SWAP gates to physically move the information.

**The Cost of Distance:**
$$1 \text{ SWAP} = 3 \text{ CNOTs}$$

This triples the error!. An AI model should ideally take the **distance matrix** of the device topology as an input. A CNOT between neighbors has error $\epsilon$; a CNOT between distant qubits has error $\approx 3 \times \text{distance} \times \epsilon$.

## 2.3 Implementation: Feature Extraction for AI

For the competition, you need to turn a quantum circuit into a mathematical vector (features) that your AI can understand.

We will write a function that transpiles the circuit to the hardware basis (`rz`, `sx`, `x`, `ecr`) and counts the operations to estimate the "noise cost".

In [ ]:
from qiskit import QuantumCircuit, transpile
# Note: In Qiskit 1.0/Stable, Aer is separate. We imported it during install.
from qiskit_aer import AerSimulator

def extract_features(qc, backend=None):
    """
    Turns a quantum circuit into a vector of numbers (features) for AI.
    """
    # 1. Transpile to hardware basis to see the TRUE cost
    # Standard IBM basis: 'rz' (virtual), 'sx' (sqrt-X), 'x', 'ecr' (entangling)
    basis_gates = ['rz', 'sx', 'x', 'ecr', 'cx', 'measure'] 
    qc_transpiled = transpile(qc, basis_gates=basis_gates, optimization_level=0)
    
    ops = qc_transpiled.count_ops()
    
    features = {
        'depth': qc_transpiled.depth(),
        'num_qubits': qc_transpiled.num_qubits,
        # High noise impact: Entangling gates (ECR or CX)
        'entangling_gates': ops.get('ecr', 0) + ops.get('cx', 0),  
        # Low noise impact: Physical single qubit gates
        'single_qubit_gates': ops.get('sx', 0) + ops.get('x', 0),  
        # Zero noise impact: Virtual gates (software only)
        'virtual_gates': ops.get('rz', 0),  
        'total_ops': sum(ops.values())
    }
    
    return features

# --- Example Usage ---
# Create a simple Bell State circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Extract and print features
features = extract_features(qc)
print("Circuit Features:", features)

# Notice: If transpiled to 'ecr', the cost is distinct from 'cx'.
# Your AI will learn that 'ecr' count correlates strongly with error.